# Setup

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

from utilities import my_show, my_gshow, my_read, my_read_g, my_read_cg

%matplotlib inline

# Contours

To compute contours, we need a binary image which we can get by applying thresholding or edge detection.  `findContours` will modify the source image (note:  as of OpenCV 3.2, this is no longer true - it does *not* modify the input image), so we will often make a copy if we need the original.

In [ ]:
messi_c, messi_g = my_read_cg('data/messi.jpg')
ret,thresh = cv2.threshold(messi_g,127,255,0)
image, contours, hierarchy = cv2.findContours(thresh,
                                              cv2.RETR_TREE,
                                              cv2.CHAIN_APPROX_NONE)

In [ ]:
contour_image = np.zeros_like(messi_c)
contour_image = cv2.drawContours(contour_image, contours, -1, (128, 128, 128), 3) # -1 means "draw all"
my_gshow(plt.gca(), contour_image)

## Working with Contours

In [ ]:
bolt = my_read_g('data/bolt.png')
my_gshow(plt.gca(), bolt)

In [ ]:
ret, thresh = cv2.threshold(bolt,127,255,cv2.THRESH_BINARY_INV)
cont_img, contours, hierarchy = cv2.findContours(thresh, 
                                                 cv2.RETR_TREE, 
                                                 cv2.CHAIN_APPROX_SIMPLE)
# various moments are defined
M = cv2.moments(contours[0])
print(M.keys())

# center of mass
cx = int(M['m10']/M['m00']) # C_x = M_{10} / M_{00}  (center of mass of x)
cy = int(M['m01']/M['m00']) # center of y
print(cx, cy)

# area
area = cv2.contourArea(contours[0])
print(area, M['m00']) # m00 is also area

# perimeter
perimeter = cv2.arcLength(contours[0], True) # True = close the curve
print(perimeter)

In [ ]:
bolt = my_read_g('data/bolt.png')
_, bolt = cv2.threshold(bolt, 127, 255, cv2.THRESH_BINARY_INV)

bolt2 = cv2.copyMakeBorder(bolt, 50,50,50,50,cv2.BORDER_CONSTANT)

bolt4 = np.tile(bolt2, (2,2))

fig, axes = plt.subplots(1,2,figsize=(8,4))
my_gshow(axes[0], bolt4)

cont_img, contours, hierarchy = cv2.findContours(bolt4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# as of opencv 3, cont_img is not modifed ... i.e., same as bolt4
my_gshow(axes[1], cont_img)

In [ ]:
# if objects are simple, len(contours) --> number of objects
# but this won't hold for real-world images
print(cont_img.shape, bolt4.shape)
print(len(contours)) 

In [ ]:
just_cont = cv2.drawContours(np.zeros_like(bolt4, dtype=np.uint8), 
                             contours, -1, (255,0,0), 5)
my_gshow(plt.gca(), just_cont)

In [ ]:
# random warning: contours overlaying an edge may be lost
cont = contours[0] # first contour object
epsilon = 0.1*cv2.arcLength(cont,True)         # error tolerance for:
approx  = cv2.approxPolyDP(cont,epsilon,True)  # smooth/simply curve via polynomials


bolt4_color = cv2.cvtColor(bolt4, cv2.COLOR_GRAY2RGB)
# WARNING:  arg[1] to drawCont MUST be a sequence OF sequences OF points;
#            if it is a sequence of points, this will break (silently) and only the first
#            point will be drawn (which you may not be able to see!)
# 3.x drawContours returns image also, but it is ref. to same object
highlight_bolt4 = cv2.drawContours(bolt4_color, [contours[0]], 0, (255,0,0), 5)
my_show(plt.gca(), highlight_bolt4)

## Contours and Outlines (Hulls, Bounding Boxes, and Bounding Circles)

In [ ]:
hull = cv2.convexHull(contours[0])
bolt4_color = cv2.cvtColor(bolt4, cv2.COLOR_GRAY2RGB)
hull_bolt4 = cv2.drawContours(bolt4_color.copy(), [hull], 0, (255,0,0), 5)
my_show(plt.gca(), hull_bolt4)

In [ ]:
print(cv2.isContourConvex(contours[0]),
      cv2.isContourConvex(hull))

In [ ]:
x,y,w,h = cv2.boundingRect(contours[0])
boxes = bolt4_color.copy()

red, green = (255,0,0), (0,255,0)
# add "perpendicular" (to axes) bounding box
boxes = cv2.rectangle(boxes, (x,y), (x+w, y+h), red, 5)

# weird return type: two points and an angle ... immediately convert
# add "rotated" (parallel to major axis) bounding box
rect = cv2.boxPoints(cv2.minAreaRect(contours[0])).astype(np.int64)
boxes = cv2.drawContours(boxes, [rect], 0, green, 5)
my_show(plt.gca(), boxes)

In [ ]:
others = bolt4_color.copy()

red, green = (255,0,0), (0,255,0)
(x,y),radius = cv2.minEnclosingCircle(contours[0])
center, radius = (int(x), int(y)), int(radius)  # really!?!
others = cv2.circle(others, center, radius, red, 5)

ellipse = cv2.fitEllipse(contours[0])
others = cv2.ellipse(others, ellipse, green, 5)
my_show(plt.gca(), others)

## Contour Statistics

In [ ]:
shape = (300, 300)
orig = np.zeros(shape, dtype=np.uint8)
orig[145:155,  50:250] = 255
orig[50:250 , 145:155] = 255

fig, axes = plt.subplots(1,2,figsize=(8,4))
my_show(axes[0], orig, cmap='gray')

cont_img, contours, hierarchy = cv2.findContours(orig.copy(),
                                                 cv2.RETR_TREE,
                                                 cv2.CHAIN_APPROX_SIMPLE)

# maximal extent
cross = contours[0].squeeze() # not a copy, drop dims of size 1
big, small = cross.argmax(axis=0), cross.argmin(axis=0)

west, north = cross[small] # upper left is (0,0)
east, south = cross[big]

cross_color = cv2.cvtColor(orig, cv2.COLOR_GRAY2RGB)
for ext in [west, north, east, south]:
    print(ext)
    cross_color = cv2.circle(cross_color, (ext[0], ext[1]), 0, (255, 0, 0), 10)
my_show(axes[1], cross_color)

In [ ]:
# various "statistics" of contours:
cont = contours[0]

x,y,w,h   = cv2.boundingRect(cont)
area      = cv2.contourArea(cont)
hull      = cv2.convexHull(cont)
hull_area = cv2.contourArea(hull)

aspect_ratio = w / h
rect_area    = w * h
extent       = area / rect_area

solidity   = area / hull_area
equiv_diam = np.sqrt(4 * area / np.pi)

orientation_angle = cv2.fitEllipse(cont)[2]

In [ ]:
mask = np.zeros_like(orig)
cv2.drawContours(mask, [cont], 0, 255, -1) #-1 means "fill me"

# r,c entries
np.transpose(np.nonzero(mask))  # r1, c2; r2, c2; ....

# to actually index these points on the original image
orig[np.nonzero(mask)] # row 1, row 2, .....     col 1, col 2, .....

In [ ]:
mask = np.zeros_like(orig)
cv2.drawContours(mask, [cont], 0, 255, -1) # fill the contour in
cv2.findNonZero(mask).shape                # "many" 1x2 things

In [ ]:
# we'll just verify that min/max vals are "on"
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(orig, mask=mask)
cv2.mean(orig, mask=mask)
# min_val, max_val, min_loc, max_loc

## Additional Contour Capabilities

In [ ]:
# hull convexity defects
hull = cv2.convexHull(cont, returnPoints = False)
defects = cv2.convexityDefects(cont,hull)

img = orig.copy()
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

# furth is the furthest point from hull segment (aka, distance to defect)
for start, end, furth, dist in defects.squeeze():
    start, end, furth = map(tuple, cont.squeeze()[[start, end, furth]])
    cv2.line(img, start, end, (255,0,0),5)
    cv2.circle(img, furth, 5, (0,255,0), -1)
my_show(plt.gca(), img)

# Template Matching

In [ ]:
messi, messi_g = my_read_cg('data/messi.jpg')

# NOTE: at row 280, col 330
ball_soi = messi_g[280:340, 330:390]

h,w = ball_soi.shape

# matching methods
#methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
#            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
matches = cv2.matchTemplate(messi_g, ball_soi, cv2.TM_CCOEFF_NORMED)
min_max = cv2.minMaxLoc(matches)

print(min_max)     
print(min_max[3])  # NOTE:  at x,y (horizontal, vertival) 330, 280
x,y = min_max[3]   #        matches are given in these terms

# drawing is done in x-y (cartesian grid) terms
cv2.rectangle(messi, (x,y), (x+h, y+h), (0,0,255), 4)
my_show(plt.gca(), messi)

In [ ]:
messi, messi_g = my_read_cg('data/messi.jpg')

ball_soi = messi_g[280:340, 330:390] # at row 280, col 330

h,w = ball_soi.shape

matches = cv2.matchTemplate(messi_g, ball_soi, cv2.TM_CCOEFF_NORMED)
THRESH = .9
rows, cols = np.where(matches > THRESH)
print(len(rows))
print(rows, cols)

# convert from r,c to x,y
x,y = cols[0], rows[0]
cv2.rectangle(messi, (x,y), (x+h, y+h), (0,0,255), 4)
my_show(plt.gca(), messi)

# Histogram Backprojection

In [ ]:
messi = my_read('data/messi.jpg')
messi_hsv = cv2.cvtColor(messi, cv2.COLOR_RGB2HSV) # HSV space

pitch = messi_hsv[280:320, 20:140].copy()

# calculating ROI histogram
pitch_hist = cv2.calcHist([pitch], [0, 1], None, [180, 256], [0, 180, 0, 256] )

# normalize histogram and apply backprojection
cv2.normalize(pitch_hist, pitch_hist, 0,255, cv2.NORM_MINMAX)
result = cv2.calcBackProject([messi_hsv], [0,1], pitch_hist, [0,180,0,256], 1)

# Now convolute with circular disc
disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
result = cv2.filter2D(result, -1, disc)

# threshold
_,thresh = cv2.threshold(result,50,255,0)
thresh = np.repeat(thresh[:,:,np.newaxis], 3, axis=2)

print(thresh.shape)
res = cv2.bitwise_and(messi_hsv,thresh)

my_show(plt.gca(), cv2.cvtColor(res, cv2.COLOR_HSV2RGB))

For comparison, we can manually achieve the same result using "raw" NumPy operations:

In [ ]:
# in "numpy" esque code
messi = my_read('data/messi.jpg')
messi_hsv = cv2.cvtColor(messi, cv2.COLOR_RGB2HSV)
pitch = messi_hsv[280:320, 20:140].copy()

M = cv2.calcHist([pitch],[0, 1], None, [180, 256], [0, 180, 0, 256] )
I = cv2.calcHist([messi_hsv],[0, 1], None, [180, 256], [0, 180, 0, 256] )

# very similar to np.interp [0,min] [255,max]
cv2.normalize(M, M, 0, 255, cv2.NORM_MINMAX)
cv2.normalize(I, I, 0, 255, cv2.NORM_MINMAX)

# conversion to pseudo-probabilities
# deal with divide by zero
R = np.minimum(M / np.where(I, I, 1E-20), 1) # could also take from M

result = R[messi_hsv[:,:,0].ravel(), 
           messi_hsv[:,:,1].ravel()].reshape(messi_hsv.shape[:2])

disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
result = cv2.filter2D(result, -1, disc).astype(np.uint8)
cv2.normalize(result, result, 0, 255, cv2.NORM_MINMAX)

# threshold and binary AND
_,thresh = cv2.threshold(result,50,255,0)
thresh   = np.repeat(thresh[:,:,np.newaxis], 3, axis=2)
res      = cv2.bitwise_and(messi_hsv, thresh)

my_show(plt.gca(), cv2.cvtColor(res, cv2.COLOR_HSV2RGB))

# Watershed

We start by basic thresholding to get outlines of our foreground:

In [ ]:
coins, coins_g = my_read_cg('data/water_coins.jpg')
_, thresh = cv2.threshold(coins_g, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

fig, axes = plt.subplots(1,3,figsize=(18,14))
my_show(axes[0], coins)
my_gshow(axes[1], coins_g)
my_gshow(axes[2], thresh)

In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(cleaned, kernel,iterations=3)

# Finding sure foreground area (regions that are far from black)
dist_transform = cv2.distanceTransform(cleaned, cv2.DIST_L2, 5)
sure_fg = cv2.threshold(dist_transform, 
                        0.7*dist_transform.max(), 255, 0)[1].astype(np.uint8)

# Finding unknown region
unknown = cv2.subtract(sure_bg, sure_fg)

fig, axes = plt.subplots(2,3, figsize=(9,6))
my_gshow(axes[0,0], cleaned)
my_gshow(axes[0,1], sure_bg)
axes[0,2].set_visible(False)

axes[0,0].set_title('Cleaned')
axes[0,1].set_title('"Sure" BG\n(in black)')

my_gshow(axes[1,0], dist_transform)
my_gshow(axes[1,1], sure_fg)
my_gshow(axes[1,2], unknown)

axes[1,0].set_title("Post-Dist X-form")
axes[1,1].set_title('"Sure" FG\n(in white)')
axes[1,2].set_title('Unknown Region')

fig.tight_layout()

With that setup, let's label background 1, the sure foreground objects 2,...,n+1, and the unknown regions 0.  Watershed will attempt to label the unknown pixels (that are currently labeled 0) with one of the positive values:  1, 2, 3, ..., n+1.  So, at the end, pixels will belong to either background or one of n objects.

In [ ]:
# known connected components (bg 0, components 1+) --> bg 1, components 2+
markers = cv2.connectedComponents(sure_fg)[1] + 1
markers[unknown==255] = 0 # enforce this brutally
obj_ct = len(set(markers.flat))

print("Found", obj_ct, "pseudo-objects (connected components)")

def magic(idx):
    ' create a custom color mapping to take marker # --> good color '
    cmap = plt.cm.tab20b if idx < 20 else plt.cm.tab20c
    idx = idx if idx < 20 else idx - 20
    return cmap(idx)
magic_table   = np.array([magic(i) for i in range(obj_ct)])

fig, axes = plt.subplots(1,3,figsize=(15,6))
my_show(axes[0], magic_table[markers])  # full array indexes back into color table

watershed = cv2.watershed(coins, markers) # apply watershed to coins from "markers" seeds 
coins[markers == -1] = [255,0,0] # ^^^ markers is modified; -1 is boundaries

my_show(axes[1], watershed)
my_show(axes[2], coins)

# Note: you might like to compare this with hough circle transform on the same image

# GrabCut

Here, we manually define a rectangle and use it to initialize grabcut.  There are a few moving parts here:
  * to keep the C code happy, we have to pre-allocate some arrays for GrabCut (`mask`, `bgm`, `fgm`)
  * the result of grabcut is an array with values `[0,1,2,3]` where
    * 0,2 represent "sure" and "probable" background and 
    * 1,3 represent "sure" and probable foreground

In [ ]:
messi = my_read('data/messi.jpg')
fig, axes = plt.subplots(1,2,figsize=(12,6))

green, gray = [0, 255, 0], [127, 127, 127]

#
# we'll manually define a rectangle to initialize grabcut
#
rect  = (50,50,450,290) # manual rectangle
messi_box = cv2.rectangle(messi.copy(), (50,50), (400, 240), green, 5)
my_show(axes[0], messi_box)

#
# setup grab cut and reprocess mask
#
# these are basically placeholder memory allocations
mask = np.zeros(messi.shape[:2], np.uint8)
bgm, fgm = [np.zeros((1,65))] * 2
mask, _, _ = cv2.grabCut(messi, mask, rect, bgm, fgm, 5, cv2.GC_INIT_WITH_RECT)

#
# In the new mask image, pixels will be marked with four values
# denoting background/foreground
#
# So we modify the mask such that all 
# 0-pixels and 2-pixels are put to 0 (ie background) and 
# 1-pixels and 3-pixels are put to 1(ie foreground pixels)
# (i.e., mapper[0] == mapper[2] == 0 (bg)
#        mapper[1] == mapper[3] == 1 (fg))
mapper = np.array([0, 1, 0, 1], dtype=np.uint8)
mask = mapper[mask]

#
# compute display results (gray background to distinguish)
#
foreground = messi * mask[:,:,np.newaxis]
background = np.full_like(messi, gray) * (1-mask[:,:,np.newaxis])
result = background + foreground

my_show(axes[1], result)

# Exercises

##### Template Matching

Use `data/mario_coins.png` and `data/mario_one_coin.png` to draw boxes around all the coins in full image.  Try using both `matchTemplate` and [fixme:  match via histogram].

In [ ]:
img, img_g = my_read_cg('data/mario_coins.png')
template   = my_read_g('data/mario_one_coin.png')

fig, axes = plt.subplots(1,3,figsize=(15,5))

# Student section here 



##### Contours, Thresholds, and Histograms
Open the apple image and find its contours.  Compare the RGB histograms of the whole image versus the histograms of the apple itself.  

In [ ]:
fig, axes = plt.subplots(3,2,figsize=(12,8))

apple, grey_apple = my_read_cg('data/apple.png')

# Student section here 



##### A Better Messi

Above, we used a manual rectangle.  But, much of the point of image processing is to do these tasks in an *automated* way.  Can you construct a pipeline that goes from the raw image, programmatically construct an initializing rectangle, and then perform grabcut?  [Hint:  get some edges, use morphology to flesh out a legitimate outline, and then calculate a bounding box to use as the rectangle.]

If you succeed with that task, you might get to a point where you get "most" of Messi with some unwanted background.  Now, try to initialize the grabcut routine with a foreground/background mask instead of a rectangle.  Note, the values for the mask array are [0,1,2,3] == "sure background", "sure foreground", "probable background", "probable foreground".  You may need to play around with how you get from a binary mask (with just two values) to your 4-value mask.

In [ ]:
messi, messi_g = my_read_cg('data/messi.jpg')
edges = cv2.Canny(messi_g, 100, 200, L2gradient=True)

fig, axes = plt.subplots(2,3, figsize=(12,6))


# Student section here 



In [ ]:
messi, messi_g = my_read_cg('data/messi.jpg')
edges = cv2.Canny(messi_g, 100, 200, L2gradient=True)

fig, axes = plt.subplots(2,2, figsize=(12,6))

# Student section here 

